In [153]:
import os

import zarr
import numpy as np
import xarray as xr
import torch
from tqdm import tqdm

from src.era5_dataset import ERA5Dataset, TimeMode
from src.fuxi_ligthning import FuXi
from torch.utils.data import DataLoader

# Batch Size darf nicht größer als 1 gewählt werden, sonst funktioniert die Logik unten beim schreiben nicht

In [154]:
bs = 1
autoregression_steps = 2
timesteps_cnt = 24
levels_cnt = 5
vars_cnt = 5
lats_cnt = 121
lons_cnt = 240
start_time = "2011-01-01T00:00:00"
end_time = "2011-01-07T18:00:00"
model_path = "/Users/xgxtphg/Documents/git/DL4WeatherAndClimate/checkpoints/epoch=0-step=2917.ckpt"



In [155]:
model = FuXi.load_from_checkpoint(model_path)
model: FuXi
model.set_autoregression_steps(autoregression_steps)
dataset = test_ds = ERA5Dataset(
        "/Users/xgxtphg/Documents/git/DL4WeatherAndClimate/data/era5_6hourly.zarr",
        TimeMode.BETWEEN,
        start_time="2011-01-01T00:00:00",
        end_time="2011-01-07T18:00:00",
        max_autoregression_steps=autoregression_steps,
        zarr_col_names='lessig'
)
dl = DataLoader(dataset, batch_size=bs, shuffle=False, num_workers=os.cpu_count()//2, pin_memory=True)


In [156]:
store = zarr.DirectoryStore('./preds.zarr')
root = zarr.group(store=store, overwrite=True)

latitude = root.create_dataset('lats', shape=(0,), chunks=(lats_cnt,), dtype=np.float64, fill_value=-99999)
levels = root.create_dataset('levels', shape=(0,), chunks=(levels_cnt,), dtype=np.int32, fill_value=-99999)
longitude = root.create_dataset('lons', shape=(0,), chunks=(lons_cnt,), dtype=np.float64, fill_value=-99999)
pred_timedelta = root.create_dataset('prediction_timedelta', shape=(0,) ,chunks=(autoregression_steps,), dtype='timedelta64[ns]')
# die Zeit muss noch richtig gesetzt werden, wahrscheinlich dann über das Dataset
time = root.create_dataset('time', shape=(0,), chunks=(timesteps_cnt,), dtype='datetime64[ns]', fill_value=-99999)

temp = root.create_dataset('temperature', shape=(0, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), dtype=np.float64,
                           chunks=(16, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), fill_value=-99999)
temp.attrs['_ARRAY_DIMENSIONS'] = ['time', 'prediction_timedelta', 'levels', 'lats', 'lons']
humid = root.create_dataset('specific_humidity', shape=(0, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), dtype=np.float64,
                            chunks=(16, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), fill_value=-99999)
humid.attrs['_ARRAY_DIMENSIONS'] = ['time', 'prediction_timedelta', 'levels', 'lats', 'lons']
uwind = root.create_dataset('u_component_of_wind', shape=(0, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), dtype=np.float64,
                            chunks=(16, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), fill_value=-99999)
uwind.attrs['_ARRAY_DIMENSIONS'] = ['time', 'prediction_timedelta', 'levels', 'lats', 'lons']
vwind = root.create_dataset('v_component_of_wind', shape=(0, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), dtype=np.float64,
                            chunks=(16, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), fill_value=-99999)
vwind.attrs['_ARRAY_DIMENSIONS'] = ['time', 'prediction_timedelta', 'levels', 'lats', 'lons']
geo = root.create_dataset('geopotential', shape=(0, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), dtype=np.float64,
                          chunks=(16, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), fill_value=-99999)
geo.attrs['_ARRAY_DIMENSIONS'] = ['time', 'prediction_timedelta', 'levels', 'lats', 'lons']


In [157]:
latitude.append(np.linspace(-90, 90, lats_cnt))
latitude.attrs['_ARRAY_DIMENSIONS'] = ['lats']

levels.append(np.arange(levels_cnt))
levels.attrs['_ARRAY_DIMENSIONS'] = ['levels']

longitude.append(np.linspace(-180, 180, lons_cnt))
longitude.attrs['_ARRAY_DIMENSIONS'] = ['lons']


timedelta = [np.timedelta64(6 * i, 'h') for i in range(autoregression_steps)]
pred_timedelta.append(np.array(timedelta))
pred_timedelta.attrs['_ARRAY_DIMENSIONS'] = ['prediction_timedelta']

times = [np.datetime64(start_time) + i*timedelta[1] for i in range(timesteps_cnt)]
time.append(times)
time.attrs['_ARRAY_DIMENSIONS'] = ['time']

In [158]:
# range muss noch durch den dataloader ersetzt werden
for idx, batch in tqdm(enumerate(dl)):
    preds = torch.reshape(model.forward(batch)[0, :, :, :, :], (-1,vars_cnt, levels_cnt, lats_cnt, lons_cnt)).numpy()
    temp.append(preds[None,:, 0, :, :, :], axis=0)
    humid.append(preds[None,:, 1, :, :, :], axis=0)
    uwind.append(preds[None,:, 2, :, :, :], axis=0)
    vwind.append(preds[None,:, 3, :, :, :], axis=0)
    geo.append(preds[None,:, 4, :, :, :], axis=0)

24it [00:08,  2.91it/s]


In [159]:
zarr.consolidate_metadata(store)

<zarr.hierarchy.Group '/'>

In [160]:
# beim laden mit xarray kommt es zum problem, dass 0´en durch nan ersetzt werden
b = xr.open_zarr('./preds.zarr')
use_dask=True
by_init=False
obs = xr.open_zarr('./preds.zarr', chunks='auto')


In [161]:
obs

<xarray.Dataset>
Dimensions:               (time: 24, prediction_timedelta: 2, levels: 5,
                           lats: 121, lons: 240)
Coordinates:
  * lats                  (lats) float64 -90.0 -88.5 -87.0 ... 87.0 88.5 90.0
  * levels                (levels) float64 0.0 1.0 2.0 3.0 4.0
  * lons                  (lons) float64 -180.0 -178.5 -177.0 ... 178.5 180.0
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 00:00:00 06:...
  * time                  (time) datetime64[ns] 2011-01-01 ... 2011-01-06T18:...
Data variables:
    geopotential          (time, prediction_timedelta, levels, lats, lons) float64 dask.array<chunksize=(16, 2, 5, 121, 240), meta=np.ndarray>
    specific_humidity     (time, prediction_timedelta, levels, lats, lons) float64 dask.array<chunksize=(16, 2, 5, 121, 240), meta=np.ndarray>
    temperature           (time, prediction_timedelta, levels, lats, lons) float64 dask.array<chunksize=(16, 2, 5, 121, 240), meta=np.ndarray>
    u_component_of_wind   (time, prediction_timedelta, levels, lats, lons) float64 dask.array<chunksize=(16, 2, 5, 121, 240), meta=np.ndarray>
    v_component_of_wind   (time, prediction_timedelta, levels, lats, lons) float64 dask.array<chunksize=(16, 2, 5, 121, 240), meta=np.ndarray>

In [162]:
obs['lats']

<xarray.DataArray 'lats' (lats: 121)>
array([-90. , -88.5, -87. , -85.5, -84. , -82.5, -81. , -79.5, -78. , -76.5,
       -75. , -73.5, -72. , -70.5, -69. , -67.5, -66. , -64.5, -63. , -61.5,
       -60. , -58.5, -57. , -55.5, -54. , -52.5, -51. , -49.5, -48. , -46.5,
       -45. , -43.5, -42. , -40.5, -39. , -37.5, -36. , -34.5, -33. , -31.5,
       -30. , -28.5, -27. , -25.5, -24. , -22.5, -21. , -19.5, -18. , -16.5,
       -15. , -13.5, -12. , -10.5,  -9. ,  -7.5,  -6. ,  -4.5,  -3. ,  -1.5,
         0. ,   1.5,   3. ,   4.5,   6. ,   7.5,   9. ,  10.5,  12. ,  13.5,
        15. ,  16.5,  18. ,  19.5,  21. ,  22.5,  24. ,  25.5,  27. ,  28.5,
        30. ,  31.5,  33. ,  34.5,  36. ,  37.5,  39. ,  40.5,  42. ,  43.5,
        45. ,  46.5,  48. ,  49.5,  51. ,  52.5,  54. ,  55.5,  57. ,  58.5,
        60. ,  61.5,  63. ,  64.5,  66. ,  67.5,  69. ,  70.5,  72. ,  73.5,
        75. ,  76.5,  78. ,  79.5,  81. ,  82.5,  84. ,  85.5,  87. ,  88.5,
        90. ])
Coordinates:
  * lats     (lats) float64 -90.0 -88.5 -87.0 -85.5 ... 85.5 87.0 88.5 90.0